# Modelo

La optimizacion la hago con el dataset undersampleado porque sino tarda una eternidad.

En el proximo script entreno ahi si con la totalidad de los datos.

In [1]:
import pandas as pd
import polars as pl
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split
from sklearn.model_selection import ShuffleSplit, StratifiedShuffleSplit
from sklearn.ensemble import RandomForestClassifier
from sklearn.impute import SimpleImputer

import lightgbm as lgb

import optuna
from optuna.visualization import plot_optimization_history, plot_param_importances, plot_slice, plot_contour

from time import time

import pickle

In [2]:
# !gsutil cp /home/eanegrin/buckets/b1/datasets/competencia_02_fe_v01.parquet /home/eanegrin/datasets/

In [2]:
# base_path = '/content/drive/MyDrive/DMEyF/2024/'
base_path = 'C:/Eugenio/Maestria/DMEyF/'
# base_path = '/home/eanegrin/buckets/b1/'

dataset_path = base_path + 'datasets/'
modelos_path = base_path + 'modelos/'
db_path = base_path + 'db/'
dataset_file = 'competencia_02_fe_v01_undersampled_10_24M_altbinaria.parquet'

ganancia_acierto = 273000
costo_estimulo = 7000

# agregue sus semillas
semillas = [122219, 109279, 400391, 401537, 999961]

# data = pd.read_parquet('/home/eanegrin/datasets/' + dataset_file)
data = pd.read_parquet(dataset_path + dataset_file)

In [3]:
meses_train = [201906, 201907, 201908, 201909, 201910, 201911, 201912,
               202001, 202003, 202004, 202005, 202006,
               202007, 202008, 202009, 202010, 202011, 202012,
               202101, 202102, 202103, 202104, 202105] # dejo afuera 202106 para test, tambien saco febrero 2020 porque caer mucho los baja+2

data = data[data['foto_mes'].isin(meses_train)]
data.shape

(361405, 679)

In [4]:
# Asignamos pesos a las clases

data['clase_peso'] = 1.0

data.loc[data['clase_ternaria'] == 'BAJA+2', 'clase_peso'] = 1.00002
data.loc[data['clase_ternaria'] == 'BAJA+1', 'clase_peso'] = 1.00001

In [5]:
X_train = data.drop(['clase_ternaria', 'clase_peso', 'clase_binaria'], axis=1)
y_train_binaria = data['clase_binaria'] # Junta a los 2 baja
w_train = data['clase_peso']

Para evaluar la calidad del modelo, crearemos nuestra propia función de evaluación que calcule la ganancia. La razón de incluir los pesos es precisamente para poder implementar esta función de evaluación de manera adecuada. Al combinar las clases *BAJA+1* y *BAJA+2* en una sola, necesitamos una forma de diferenciarlas, y es aquí donde entra en juego el *weight*. Este parámetro nos permitirá distinguir entre ambas clases al momento de evaluarlas dentro del algoritmo.


In [6]:
def lgb_gan_eval(y_pred, data):
    weight = data.get_weight()
    ganancia = np.where(weight == 1.00002, ganancia_acierto, 0) - np.where(weight < 1.00002, costo_estimulo, 0)
    ganancia = ganancia[np.argsort(y_pred)[::-1]]
    ganancia = np.cumsum(ganancia)

    return 'gan_eval', np.max(ganancia) , True

# Optimizacion

In [7]:
def objective(trial):

    num_leaves = trial.suggest_int('num_leaves', 8, 150), # segun statquest esto deberia ir de 8 a 32
    learning_rate = trial.suggest_float('learning_rate', 0.005, 0.05), # mas bajo, más iteraciones necesita
    min_data_in_leaf = trial.suggest_int('min_data_in_leaf', 1, 2000),
    feature_fraction = trial.suggest_float('feature_fraction', 0.1, 1.0),
    bagging_fraction = trial.suggest_float('bagging_fraction', 0.1, 1.0),

    params = {
        'objective': 'binary',
        'metric': 'custom',
        'boosting_type': 'gbdt',
        'first_metric_only': True,
        'boost_from_average': True,
        'feature_pre_filter': False,
        'max_bin': 31,
        'num_leaves': num_leaves,
        'learning_rate': learning_rate,
        'min_data_in_leaf': min_data_in_leaf,
        'feature_fraction': feature_fraction,
        'bagging_fraction': bagging_fraction,
        'seed': semillas[0],
        'verbose': -1
    }
    
    train_data = lgb.Dataset(X_train,
                              label=y_train_binaria, # eligir la clase
                              weight=w_train)
    
    # print(f"Learning Rate: {learning_rate}, Type: {type(learning_rate)}")
    
    # Use callbacks for early stopping
    early_stopping_cb = lgb.early_stopping(stopping_rounds=50) # creo que en min_delta le tendria que pasar un parametro con la ganancia que considero irrelevante
        
    cv_results = lgb.cv(
        params,
        train_data,
        num_boost_round=1000, # modificar, subit y subir... y descomentar la línea inferior (ahora le puso 100 para mostrarnos, pero hay que ponerle un numero alto, 10.000, 200.000)
        callbacks=[early_stopping_cb],
        feval=lgb_gan_eval,
        stratified=True,
        nfold=5,
        seed=semillas[0]
    )
    
    max_gan = max(cv_results['valid gan_eval-mean'])
    best_iter = cv_results['valid gan_eval-mean'].index(max_gan) + 1

    # Guardamos cual es la mejor iteración del modelo
    trial.set_user_attr("best_iter", best_iter)

    return max_gan * 5 # funcion objetivo, en el proximo paso le digo si quiero maximizarla o minimizarla.

In [8]:
storage_name = "sqlite:///" + db_path + "optimization_lgbm.db"
study_name = "competencia2_lgbm_v09" # UPDATE

study = optuna.create_study(
    direction="maximize",
    study_name=study_name,
    storage=storage_name,
    load_if_exists=True,
)

[I 2024-11-19 23:23:41,269] Using an existing study with name 'competencia2_lgbm_v09' instead of creating a new one.


In [12]:
study.optimize(objective, n_trials=100) # Habia hecho 30, hago 70 mas porque parece que es un modelo superior

Training until validation scores don't improve for 50 rounds
Early stopping, best iteration is:
[478]	cv_agg's valid gan_eval: 6.53633e+08 + 2.62019e+06


[I 2024-11-19 23:27:26,415] Trial 0 finished with value: 3268167000.0 and parameters: {'num_leaves': 150, 'learning_rate': 0.018864010768457368, 'min_data_in_leaf': 742, 'feature_fraction': 0.37203860717154613, 'bagging_fraction': 0.9375365716577849}. Best is trial 0 with value: 3268167000.0.


Training until validation scores don't improve for 50 rounds
Early stopping, best iteration is:
[569]	cv_agg's valid gan_eval: 6.53848e+08 + 3.26316e+06


[I 2024-11-19 23:31:36,513] Trial 1 finished with value: 3269238000.0 and parameters: {'num_leaves': 78, 'learning_rate': 0.017990732842157613, 'min_data_in_leaf': 937, 'feature_fraction': 0.775096985591943, 'bagging_fraction': 0.21893335902273792}. Best is trial 1 with value: 3269238000.0.


Training until validation scores don't improve for 50 rounds
Early stopping, best iteration is:
[552]	cv_agg's valid gan_eval: 6.54308e+08 + 3.3061e+06


...

[I 2024-11-20 04:40:21,215] Trial 97 finished with value: 3276238000.0 and parameters: {'num_leaves': 138, 'learning_rate': 0.011254802294562802, 'min_data_in_leaf': 967, 'feature_fraction': 0.48596117163928226, 'bagging_fraction': 0.6791904589266702}. Best is trial 71 with value: 3281369000.0.


Training until validation scores don't improve for 50 rounds
Early stopping, best iteration is:
[759]	cv_agg's valid gan_eval: 6.54489e+08 + 4.39043e+06


[I 2024-11-20 04:44:58,961] Trial 98 finished with value: 3272444000.0 and parameters: {'num_leaves': 132, 'learning_rate': 0.012343875628665521, 'min_data_in_leaf': 1726, 'feature_fraction': 0.4924984758562308, 'bagging_fraction': 0.7448180304036599}. Best is trial 71 with value: 3281369000.0.


Training until validation scores don't improve for 50 rounds
Early stopping, best iteration is:
[509]	cv_agg's valid gan_eval: 6.53929e+08 + 3.93332e+06


[I 2024-11-20 04:48:26,057] Trial 99 finished with value: 3269644000.0 and parameters: {'num_leaves': 138, 'learning_rate': 0.013817634580729432, 'min_data_in_leaf': 1078, 'feature_fraction': 0.545139611970267, 'bagging_fraction': 0.5925128291803816}. Best is trial 71 with value: 3281369000.0.


Analizamos los resultados as usual

In [13]:
optuna.visualization.plot_optimization_history(study)

In [14]:
plot_param_importances(study)

El **learning rate** es un parámetro que tiene que ir acompañado por más árboles.

In [15]:
plot_slice(study)

In [16]:
plot_contour(study)

In [17]:
plot_contour(study, params=['num_leaves','min_data_in_leaf'])

In [18]:
study.best_trial.params

{'num_leaves': 137,
 'learning_rate': 0.01629629542229838,
 'min_data_in_leaf': 794,
 'feature_fraction': 0.3718551056343741,
 'bagging_fraction': 0.7866731437540004}

In [19]:
best_iter = study.best_trial.user_attrs["best_iter"]
best_iter

638